# Backtest multiple models
Loads valuation model(s) and applies error handling to rank sectors and trade

In [84]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from keras import layers
import pandas_datareader as pdr
from datetime import datetime
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import itertools
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform



In [2]:

NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 1
DAY_OFFSET = 5

def from_network(symbol):
    return pdr.get_data_yahoo(symbols=symbol, start=datetime(1900, 1, 1)).sort_values(by=['Date'],ascending=False)

# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation
    return map(lambda arr: map(lambda val: round(val,4),arr), labels)

def convert_to_train(raw_dataset, offset=5):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(offset, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]

def split_data(symbol):
    fetched = from_network(symbol)
    converted = convert_to_train(fetched)
    features = converted[0]
    labels = converted[1]
    training = [ # since we decide a model, use all data for training
        features[1:],
        labels[1:]
    ]
    validation = [
        features[5:1000],
        labels[5:1000]
    ]
    prediction = [
        features[:5],
        labels[:5],
    ]
    
    return {
        'symbol': symbol,
        'prediction': prediction,
        'validation': validation,
        'training': training,
    }

    
def combine_all(accum, prep):
    return {   
        'prediction':[
            accum['prediction'][0] + prep['prediction'][0],
            accum['prediction'][1] + prep['prediction'][1],
        ],
        'validation':[
            accum['validation'][0] + prep['validation'][0],
            accum['validation'][1] + prep['validation'][1],
        ],
        'training':[
            accum['training'][0] + prep['training'][0],
            accum['training'][1] + prep['training'][1],
        ],
    }


In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    error_model = load_model('sector_model_error.h5')
    model = load_model('sector_model.h5')
    model_0 = load_model('sector_model_0.h5')
    model_1 = load_model('sector_model_1.h5')
    model_2 = load_model('sector_model_2.h5')
    model_3 = load_model('sector_model_3.h5')
    model_4 = load_model('sector_model_4.h5')
    model_5 = load_model('sector_model_5.h5')
    model_6 = load_model('sector_model_6.h5')
    model_7 = load_model('sector_model_7.h5')
    model_8 = load_model('sector_model_8.h5')
    model_9 = load_model('sector_model_9.h5')


In [4]:

QQQ = from_network('QQQ')
SPY = from_network('SPY')
XLK = from_network('XLK')
XLF = from_network('XLF')
XLE = from_network('XLE')
XLP = from_network('XLP')
XLV = from_network('XLV')
XLY = from_network('XLY')
XLI = from_network('XLI')
XLU = from_network('XLU')
DIA = from_network('DIA')
IWM = from_network('IWM')

In [5]:
securities_to_predict = [
    ['SPY', np.array(convert_to_train(SPY.copy(), 0)[0][:1])],
#     ['QQQ', np.array(convert_to_train(QQQ.copy(), 0)[0][:1])],
    ['XLE', np.array(convert_to_train(XLE.copy(), 0)[0][:1])],
    ['XLF', np.array(convert_to_train(XLF.copy(), 0)[0][:1])],
    ['XLK', np.array(convert_to_train(XLK.copy(), 0)[0][:1])],
    ['XLP', np.array(convert_to_train(XLP.copy(), 0)[0][:1])],
    ['XLV', np.array(convert_to_train(XLV.copy(), 0)[0][:1])],
    ['XLU', np.array(convert_to_train(XLU.copy(), 0)[0][:1])],
    ['XLY', np.array(convert_to_train(XLY.copy(), 0)[0][:1])],
    ['XLI', np.array(convert_to_train(XLI.copy(), 0)[0][:1])],
#     ['IWM', np.array(convert_to_train(IWM.copy(), 0)[0][:1])],
#     ['DIA', np.array(convert_to_train(DIA.copy(), 0)[0][:1])],

]


In [33]:
map(generate_model_predictions,[model, model_1, model_2, model_3, model_4,model_5,model_6,model_7,model_8,model_9,model_0])

['XLK', 'XLE', 'XLP', 'XLY', 'SPY', 'XLV', 'XLF', 'XLI', 'XLU']
['XLK', 'XLI', 'XLF', 'XLY', 'SPY', 'XLU', 'XLE', 'XLV', 'XLP']
['XLE', 'XLK', 'XLU', 'XLP', 'XLI', 'XLV', 'SPY', 'XLF', 'XLY']
['XLI', 'XLY', 'XLV', 'XLE', 'XLP', 'SPY', 'XLU', 'XLK', 'XLF']
['XLE', 'XLY', 'XLK', 'XLI', 'XLU', 'XLV', 'SPY', 'XLP', 'XLF']
['XLU', 'XLP', 'XLV', 'XLI', 'SPY', 'XLF', 'XLK', 'XLY', 'XLE']
['XLE', 'XLK', 'XLP', 'XLF', 'XLV', 'XLU', 'XLY', 'XLI', 'SPY']
['XLY', 'XLP', 'XLI', 'SPY', 'XLU', 'XLF', 'XLV', 'XLK', 'XLE']
['SPY', 'XLI', 'XLF', 'XLK', 'XLY', 'XLV', 'XLP', 'XLE', 'XLU']
['XLE', 'XLP', 'XLK', 'XLV', 'XLY', 'SPY', 'XLU', 'XLF', 'XLI']
['XLK', 'XLI', 'XLF', 'XLY', 'SPY', 'XLP', 'XLU', 'XLE', 'XLV']


[None, None, None, None, None, None, None, None, None, None, None]

In [47]:
SPY_TRAIN = convert_to_train(SPY.copy(), 0)
XLE_TRAIN = convert_to_train(XLE.copy(), 0)
XLF_TRAIN = convert_to_train(XLF.copy(), 0)
XLK_TRAIN = convert_to_train(XLK.copy(), 0)
XLP_TRAIN = convert_to_train(XLP.copy(), 0)
XLV_TRAIN = convert_to_train(XLV.copy(), 0)
XLU_TRAIN = convert_to_train(XLU.copy(), 0)
XLY_TRAIN = convert_to_train(XLY.copy(), 0)
XLI_TRAIN = convert_to_train(XLI.copy(), 0)


```
Perhaps creating a policy which takes the aggregate score 0-1 to create a normalized ranking for an item and trading it as a softmax of the portfolio?
```

In [209]:
account = 1
for x, y in zip(range(5,60), itertools.count()):
    securities_to_predict = [
#         ['SPY', np.array(SPY_TRAIN[0][x:x+1]),np.array(SPY_TRAIN[1][x:x+1])],
        ['XLE', np.array(XLE_TRAIN[0][x:x+1]),np.array(XLE_TRAIN[1][x:x+1])],
        ['XLF', np.array(XLF_TRAIN[0][x:x+1]),np.array(XLF_TRAIN[1][x:x+1])],
        ['XLK', np.array(XLK_TRAIN[0][x:x+1]),np.array(XLK_TRAIN[1][x:x+1])],
        ['XLP', np.array(XLP_TRAIN[0][x:x+1]),np.array(XLP_TRAIN[1][x:x+1])],
        ['XLV', np.array(XLV_TRAIN[0][x:x+1]),np.array(XLV_TRAIN[1][x:x+1])],
        ['XLU', np.array(XLU_TRAIN[0][x:x+1]),np.array(XLU_TRAIN[1][x:x+1])],
        ['XLY', np.array(XLY_TRAIN[0][x:x+1]),np.array(XLY_TRAIN[1][x:x+1])],
        ['XLI', np.array(XLI_TRAIN[0][x:x+1]),np.array(XLI_TRAIN[1][x:x+1])],
    ]

    def print_arg(arr):
        print '{} {}'.format(arr[0],str(arr[1]))


    def predict_and_correct(model, inputs):
        prediction = model.predict(inputs)
        return prediction

    def generate_model_predictions(model):
        predictions = map(lambda arr: [arr[0], predict_and_correct(model,arr[1])], securities_to_predict)
        predictions.sort(key=lambda x: x[1],reverse=True)
#         print map(print_arg, predictions)
#         print map(lambda arr: arr[0],predictions)
        return predictions
    
    all_model_predictions = map(generate_model_predictions,[model, model_1, model_2, model_3, model_4,model_5,model_6,model_7,model_8,model_9,model_0])

    def ranking_for_symbol(symbol):
        return map(lambda predictions: map(lambda arr: arr[0],predictions).index(symbol),all_model_predictions)

    securities_to_predict.sort(key=lambda x: np.median(ranking_for_symbol(x[0])))

    print 'result ranking'
    print map(lambda x: x[0],securities_to_predict)
    for security in securities_to_predict:
        print security[0],security[2],np.median(ranking_for_symbol(security[0])),ranking_for_symbol(security[0])
    buy = np.mean(map(lambda x: x[2],
                      filter(lambda x: np.median(ranking_for_symbol(x[0])) <= 2, securities_to_predict[:4]))) # long
    sell = 1+(1-np.mean(map(lambda x: x[2],
                            filter(lambda x: np.median(ranking_for_symbol(x[0])) >= 7, securities_to_predict[4:])))) # short
    buy = 1 if np.isnan(buy) else buy
    sell = 1 if np.isnan(sell) else sell

    print 'Buy',buy
    print 'Sell',sell
    change = ((buy + sell) / 2) - 1
    if change != 0:
        print 'Change', change
    account = account + (change)
    print 'Account', account
print 'End Account', account



result ranking
['XLK', 'XLE', 'XLF', 'XLU', 'XLP', 'XLV', 'XLY', 'XLI']
XLK [[1.0166]] 1.0 [1, 0, 6, 7, 0, 7, 0, 5, 1, 5, 0]
XLE [[0.9988]] 2.0 [0, 1, 3, 0, 2, 6, 2, 4, 7, 0, 4]
XLF [[1.0161]] 3.0 [5, 3, 1, 5, 3, 3, 7, 7, 2, 4, 2]
XLU [[1.0125]] 3.0 [2, 6, 2, 1, 4, 1, 6, 3, 6, 3, 6]
XLP [[1.009]] 4.0 [3, 5, 7, 2, 1, 0, 4, 6, 5, 2, 7]
XLV [[1.0129]] 4.0 [7, 7, 4, 3, 5, 2, 3, 0, 4, 1, 5]
XLY [[1.0152]] 4.0 [4, 4, 0, 6, 6, 5, 1, 1, 0, 7, 1]
XLI [[1.0151]] 4.0 [6, 2, 5, 4, 7, 4, 5, 2, 3, 6, 3]
Buy 1.0077
Sell 1
Change 0.003849999999999909
Account 1.00385
result ranking
['XLE', 'XLF', 'XLY', 'XLI', 'XLK', 'XLP', 'XLV', 'XLU']
XLE [[0.9702]] 3.0 [3, 5, 1, 6, 7, 7, 2, 3, 2, 3, 5]
XLF [[0.9667]] 3.0 [2, 4, 3, 0, 4, 2, 6, 2, 6, 0, 3]
XLY [[0.9732]] 3.0 [1, 3, 0, 7, 0, 6, 1, 6, 7, 7, 2]
XLI [[0.9723]] 3.0 [6, 1, 6, 5, 2, 1, 3, 1, 3, 6, 0]
XLK [[0.9583]] 4.0 [0, 2, 2, 4, 6, 3, 0, 5, 5, 4, 4]
XLP [[0.974]] 4.0 [7, 0, 7, 1, 1, 0, 5, 0, 4, 5, 6]
XLV [[0.9765]] 4.0 [5, 6, 5, 2, 5, 4, 4, 4, 1, 1, 1]
X

result ranking
['XLU', 'XLE', 'XLV', 'XLI', 'XLF', 'XLK', 'XLY', 'XLP']
XLU [[1.004]] 1.0 [1, 0, 3, 0, 1, 5, 7, 0, 5, 5, 1]
XLE [[0.991]] 2.0 [6, 1, 7, 1, 0, 0, 2, 7, 7, 2, 0]
XLV [[1.0022]] 3.0 [0, 5, 1, 3, 4, 1, 5, 3, 0, 0, 3]
XLI [[0.9959]] 3.0 [3, 3, 6, 2, 6, 2, 1, 6, 4, 4, 2]
XLF [[0.9944]] 4.0 [4, 6, 0, 5, 7, 3, 3, 5, 2, 1, 4]
XLK [[1.0033]] 4.0 [7, 2, 5, 7, 2, 7, 0, 4, 1, 3, 7]
XLY [[1.0032]] 4.0 [5, 4, 4, 6, 3, 6, 4, 2, 3, 7, 6]
XLP [[1.0023]] 5.0 [2, 7, 2, 4, 5, 4, 6, 1, 6, 6, 5]
Buy 0.9975
Sell 1
Change -0.0012499999999999734
Account 0.99515
result ranking
['XLE', 'XLK', 'XLV', 'XLU', 'XLI', 'XLY', 'XLF', 'XLP']
XLE [[1.0042]] 0.0 [4, 0, 7, 0, 0, 0, 0, 7, 4, 0, 0]
XLK [[1.0073]] 2.0 [7, 1, 4, 2, 2, 6, 2, 0, 0, 6, 6]
XLV [[0.9889]] 3.0 [1, 6, 3, 6, 1, 3, 6, 3, 3, 1, 1]
XLU [[0.9934]] 3.0 [0, 3, 0, 5, 4, 2, 3, 1, 5, 5, 3]
XLI [[1.0175]] 3.0 [3, 4, 5, 3, 5, 1, 1, 5, 7, 3, 2]
XLY [[1.0115]] 4.0 [6, 2, 6, 1, 3, 4, 5, 2, 1, 7, 4]
XLF [[1.0068]] 5.0 [5, 5, 2, 4, 7, 7, 4, 6, 2, 2, 7]

XLE [[0.991]] 6.0 [7, 7, 5, 7, 4, 2, 7, 6, 0, 4, 7]
Buy 0.9975666666666667
Sell 1
Change -0.0012166666666666437
Account 1.0117916666666662
result ranking
['XLV', 'XLF', 'XLK', 'XLU', 'XLI', 'XLP', 'XLY', 'XLE']
XLV [[1.0005]] 2.0 [1, 2, 2, 5, 1, 3, 5, 2, 1, 0, 5]
XLF [[0.9927]] 3.0 [3, 0, 0, 0, 7, 7, 2, 3, 6, 1, 3]
XLK [[0.9918]] 3.0 [2, 1, 6, 2, 3, 6, 1, 7, 7, 6, 0]
XLU [[0.9959]] 3.0 [0, 5, 5, 6, 2, 1, 6, 0, 3, 3, 6]
XLI [[0.9904]] 3.0 [5, 3, 3, 1, 6, 2, 3, 4, 5, 7, 2]
XLP [[0.9892]] 4.0 [6, 7, 4, 4, 5, 5, 4, 1, 2, 2, 7]
XLY [[0.9919]] 4.0 [4, 4, 1, 3, 4, 4, 0, 5, 4, 5, 1]
XLE [[0.9981]] 6.0 [7, 6, 7, 7, 0, 0, 7, 6, 0, 4, 4]
Buy 1.0005
Sell 1
Change 0.00024999999999986144
Account 1.012041666666666
result ranking
['XLF', 'XLE', 'XLU', 'XLI', 'XLP', 'XLV', 'XLY', 'XLK']
XLF [[1.0044]] 2.0 [2, 0, 1, 0, 4, 5, 3, 2, 6, 1, 6]
XLE [[1.0222]] 3.0 [7, 4, 0, 4, 7, 0, 7, 0, 2, 0, 3]
XLU [[1.0064]] 3.0 [3, 6, 3, 7, 1, 1, 6, 1, 1, 3, 5]
XLI [[1.0167]] 3.0 [0, 1, 2, 3, 6, 4, 2, 4, 4, 5, 2]
XLP [[1

result ranking
['XLE', 'XLK', 'XLU', 'XLP', 'XLF', 'XLY', 'XLV', 'XLI']
XLE [[0.9824]] 2.0 [1, 2, 0, 5, 0, 7, 6, 0, 6, 2, 3]
XLK [[0.9833]] 2.0 [5, 5, 7, 1, 2, 0, 2, 3, 1, 0, 6]
XLU [[1.0047]] 2.0 [3, 1, 3, 0, 1, 2, 5, 1, 2, 6, 5]
XLP [[0.9858]] 3.0 [0, 3, 4, 3, 5, 5, 3, 2, 3, 3, 1]
XLF [[0.9849]] 4.0 [4, 7, 2, 6, 7, 3, 0, 6, 4, 1, 0]
XLY [[0.9869]] 4.0 [6, 4, 6, 2, 3, 4, 1, 5, 0, 4, 2]
XLV [[0.9928]] 5.0 [2, 0, 1, 7, 6, 6, 7, 4, 5, 7, 4]
XLI [[0.9863]] 5.0 [7, 6, 5, 4, 4, 1, 4, 7, 7, 5, 7]
Buy 0.9901333333333332
Sell 1
Change -0.004933333333333456
Account 1.0431083333333329
result ranking
['XLY', 'XLE', 'XLP', 'XLF', 'XLK', 'XLV', 'XLU', 'XLI']
XLY [[1.0065]] 2.0 [6, 2, 6, 7, 1, 0, 0, 2, 1, 2, 6]
XLE [[0.9873]] 3.0 [3, 0, 0, 5, 4, 2, 6, 0, 3, 0, 7]
XLP [[1.0043]] 3.0 [2, 5, 4, 1, 5, 7, 3, 3, 0, 3, 0]
XLF [[0.9966]] 4.0 [1, 7, 2, 3, 7, 5, 1, 4, 6, 4, 1]
XLK [[1.0062]] 4.0 [7, 4, 7, 6, 0, 1, 4, 6, 2, 1, 4]
XLV [[1.0055]] 4.0 [4, 1, 1, 2, 6, 3, 7, 5, 7, 7, 3]
XLU [[0.9985]] 4.0 [5, 3, 3,

In [210]:
print 'End Account', account


End Account 1.0314833333333329
